In [1]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features = ['ADAS_Q1', 'ADAS_Q4', 'LDELTOTAL', 'AVDEL30MIN','tmab_time', 
           'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
           'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg', 'faq1','faq2','faq3','faq4','faq5','faq6','faq7','faq8','faq9','faq10']

categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']

In [2]:
_ = features
print(len(_))
print('[%s]' % ', '.join(map(str, _)))

27
[ADAS_Q1, ADAS_Q4, LDELTOTAL, AVDEL30MIN, tmab_time, e_memory_pt, e_lang_pt, e_visspat_pt, e_plan_pt, e_organ_pt, e_divatt_pt, e_memory_cg, e_lang_cg, e_visspat_cg, e_plan_cg, e_organ_cg, e_divatt_cg, faq1, faq2, faq3, faq4, faq5, faq6, faq7, faq8, faq9, faq10]


In [2]:
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(4)),repeats=list(int(x)*42+42 for x in range(5)), num_classes=3)
df.describe()

Wall time: 5.12 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.845455,0.918519,0.885714,0.881987,0.864583,0.821500,0.842602,0.843042,0.645833,0.977174,0.956888,0.811504,0.785290,0.905731,0.895068,0.944464
std,0.037671,0.014900,0.014657,0.016437,0.024604,0.028149,0.015553,0.015485,0.155021,0.010068,0.012873,0.077289,0.046495,0.008725,0.010368,0.007346
min,0.772727,0.898148,0.857143,0.850379,0.822917,0.780000,0.816327,0.817083,0.416667,0.956522,0.933673,0.700181,0.707702,0.889831,0.877551,0.932861
25%,0.829545,0.907407,0.877551,0.873895,0.854167,0.800000,0.831633,0.831771,0.562500,0.972826,0.948980,0.763587,0.759785,0.898761,0.887755,0.938305
50%,0.852273,0.916667,0.887755,0.884996,0.864583,0.820000,0.841837,0.842187,0.666667,0.978261,0.959184,0.819746,0.783460,0.906272,0.894558,0.942449
75%,0.875000,0.921296,0.897959,0.895833,0.875000,0.842500,0.853316,0.853542,0.750000,0.983696,0.965561,0.869565,0.828046,0.911201,0.902211,0.949291
max,0.886364,0.953704,0.908163,0.902988,0.927083,0.870000,0.872449,0.872500,0.833333,0.989130,0.974490,0.903080,0.838068,0.922179,0.914966,0.957352


## Random Search

In [5]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   45.1s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  6.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=0.5,
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                  

In [7]:
rf_search_rand.best_params_

{'n_estimators': 2000,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 0.5,
 'max_depth': 20,
 'bootstrap': True}

In [8]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(3)),repeats=list(int(x)*42+42 for x in range(3)), num_classes=3)
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.837121,0.918724,0.882086,0.877923,0.876157,0.816667,0.845805,0.846412,0.666667,0.983092,0.963719,0.824879,0.793315,0.906161,0.897203,0.947692
std,0.039773,0.015199,0.022594,0.023999,0.019795,0.026458,0.018707,0.018621,0.190941,0.005729,0.011535,0.094769,0.050155,0.010639,0.012472,0.010760
min,0.784091,0.907407,0.852041,0.845749,0.854167,0.790000,0.821429,0.822083,0.416667,0.972826,0.948980,0.700181,0.725379,0.891889,0.880952,0.933435
25%,0.784091,0.907407,0.857143,0.850379,0.854167,0.790000,0.821429,0.822083,0.416667,0.978261,0.948980,0.700181,0.728851,0.893164,0.880952,0.935820
50%,0.863636,0.907407,0.887755,0.885522,0.875000,0.810000,0.852041,0.852917,0.750000,0.983696,0.969388,0.869565,0.823864,0.909627,0.901361,0.949376
75%,0.863636,0.935185,0.903061,0.899411,0.895833,0.850000,0.862245,0.862500,0.833333,0.989130,0.974490,0.903080,0.829545,0.915513,0.908163,0.958536
max,0.863636,0.944444,0.908163,0.904040,0.906250,0.850000,0.862245,0.862500,0.833333,0.989130,0.974490,0.905797,0.829545,0.915513,0.908163,0.959794


## Grid Search

In [9]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [1900, 2000, 2100],
               'max_features': [.4, .5, .6],
               'max_depth': [10, 20, 30],
               'min_samples_split': [8, 9, 10],
               'min_samples_leaf': [3, 4, 5],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed: 14.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=0.5,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=210,
                                    

In [10]:
rf_search_grid.best_params_

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 0.5,
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 2100}

In [11]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(3)),repeats=list(int(x)*42+42 for x in range(3)), num_classes=3)
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.838384,0.915638,0.880952,0.877011,0.871528,0.815556,0.842971,0.843542,0.648148,0.982488,0.962018,0.815318,0.786020,0.904560,0.895314,0.947337
std,0.040885,0.015662,0.022385,0.023871,0.020172,0.029202,0.020373,0.020258,0.175682,0.006532,0.011440,0.087345,0.046458,0.011514,0.013582,0.010473
min,0.784091,0.898148,0.852041,0.845749,0.843750,0.780000,0.811224,0.811875,0.416667,0.972826,0.948980,0.700181,0.725379,0.886744,0.874150,0.934122
25%,0.784091,0.907407,0.857143,0.850379,0.854167,0.790000,0.821429,0.822083,0.416667,0.978261,0.948980,0.700181,0.729167,0.891889,0.880952,0.935691
50%,0.863636,0.907407,0.882653,0.880892,0.864583,0.810000,0.852041,0.852917,0.750000,0.983696,0.959184,0.861413,0.796086,0.909627,0.901361,0.948841
75%,0.863636,0.935185,0.903061,0.899411,0.895833,0.850000,0.857143,0.857292,0.750000,0.989130,0.974490,0.869565,0.826073,0.912426,0.904762,0.958093
max,0.875000,0.935185,0.908163,0.905093,0.895833,0.850000,0.862245,0.862500,0.833333,0.989130,0.974490,0.905797,0.829545,0.915513,0.908163,0.959280


In [12]:
rf = RandomForestClassifier(n_estimators=2100, max_features=.5, max_depth = 10, min_samples_split = 10, min_samples_leaf = 3, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(4)),repeats=list(int(x)*42+42 for x in range(5)), num_classes=3)
df.describe()

Wall time: 1min 23s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.840909,0.919907,0.884439,0.880408,0.875000,0.816500,0.845153,0.845750,0.637500,0.981793,0.960714,0.809647,0.784470,0.906067,0.896769,0.946498
std,0.034190,0.015399,0.019952,0.021045,0.018817,0.023902,0.016984,0.016917,0.155961,0.005921,0.010083,0.077466,0.040772,0.009710,0.011323,0.008736
min,0.784091,0.898148,0.852041,0.845749,0.843750,0.780000,0.811224,0.811875,0.416667,0.972826,0.948980,0.700181,0.725063,0.886744,0.874150,0.934122
25%,0.835227,0.907407,0.876276,0.873264,0.861979,0.797500,0.840561,0.841146,0.541667,0.978261,0.954082,0.761322,0.759943,0.904019,0.893707,0.941773
50%,0.852273,0.921296,0.890306,0.887311,0.875000,0.820000,0.852041,0.852813,0.666667,0.980978,0.956633,0.821105,0.783144,0.910388,0.901361,0.946107
75%,0.863636,0.935185,0.897959,0.893729,0.888021,0.827500,0.857143,0.857292,0.750000,0.989130,0.970663,0.869565,0.824416,0.912426,0.904762,0.951263
max,0.875000,0.935185,0.908163,0.905093,0.906250,0.850000,0.862245,0.862500,0.833333,0.989130,0.974490,0.905797,0.829545,0.915513,0.908163,0.959280


In [2]:
# All samples
rf = RandomForestClassifier(n_estimators=2100, max_features=.5, max_depth = 10, min_samples_split = 10, min_samples_leaf = 3, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), num_classes=3, imbalanced='over', avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 1h 7s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.866231,0.915534,0.893733,0.890882,0.858129,0.851395,0.854762,0.854762,0.737941,0.974720,0.961029,0.856331,0.854762,0.852999,0.854762,0.944722
std,0.027773,0.019225,0.015829,0.016425,0.032476,0.025816,0.020297,0.020297,0.080927,0.010941,0.011292,0.040794,0.020297,0.020057,0.020297,0.009744
min,0.792308,0.871951,0.853741,0.847373,0.768707,0.795918,0.802721,0.802721,0.411765,0.942238,0.925170,0.684222,0.802721,0.796975,0.802721,0.926983
25%,0.853846,0.902439,0.884354,0.881191,0.835034,0.843537,0.840136,0.840136,0.705882,0.967509,0.959184,0.842111,0.840136,0.840131,0.840136,0.937698
50%,0.873077,0.914634,0.887755,0.887430,0.857143,0.850340,0.853741,0.853741,0.764706,0.978339,0.962585,0.857082,0.853741,0.851305,0.853741,0.944917
75%,0.884615,0.932927,0.908163,0.906027,0.884354,0.870748,0.874150,0.874150,0.764706,0.981949,0.969388,0.876938,0.874150,0.871660,0.874150,0.951137
max,0.915385,0.945122,0.925170,0.922561,0.911565,0.904762,0.891156,0.891156,0.882353,0.989170,0.976190,0.932151,0.891156,0.891032,0.891156,0.965574


# Outputs

In [3]:
### Save outputs ###
q = 'CDR_MultiC_BorutaECOGFAQ_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)